# Mnist on Jupysterlab

1. Launch jpyterlab with hasktorch.
   > docker run --gpus all -it --rm -p 8888:8888 htorch/hasktorch-jupyter:latest-cu10
2. Download mnist files.
3. Train a MLP model.


In [7]:
:! wget -q https://data.deepai.org/mnist.zip
:! unzip mnist.zip
:! mkdir data
:! mv *.gz data/

Archive:  mnist.zip
  inflating: train-labels-idx1-ubyte.gz  
  inflating: train-images-idx3-ubyte.gz  
  inflating: t10k-images-idx3-ubyte.gz  
  inflating: t10k-labels-idx1-ubyte.gz

In [6]:
{-# LANGUAGE DeriveAnyClass #-}
{-# LANGUAGE DeriveGeneric #-}
{-# LANGUAGE FlexibleContexts #-}
{-# LANGUAGE MultiParamTypeClasses #-}
{-# LANGUAGE RecordWildCards #-}
{-# LANGUAGE ScopedTypeVariables #-}
{-# LANGUAGE TypeApplications #-}


import Control.Exception.Safe
  ( SomeException (..),
    try,
  )
import Control.Monad (forM_, when, (<=<))
import Control.Monad.Cont (ContT (..))
import GHC.Generics
import Pipes
import qualified Pipes.Prelude as P
import System.Environment
import Torch
import Torch.Tensor
import Torch.Lens
import Torch.Serialize
import Torch.Typed.Vision (initMnist)
import qualified Torch.Vision as V
import Prelude hiding (exp)

data MLPSpec = MLPSpec
  { inputFeatures :: Int,
    hiddenFeatures0 :: Int,
    hiddenFeatures1 :: Int,
    outputFeatures :: Int
  }
  deriving (Show, Eq)

data MLP = MLP
  { l0 :: Linear,
    l1 :: Linear,
    l2 :: Linear
  }
  deriving (Generic, Show, Parameterized)

instance Randomizable MLPSpec MLP where
  sample MLPSpec {..} =
    MLP
      <$> sample (LinearSpec inputFeatures hiddenFeatures0)
      <*> sample (LinearSpec hiddenFeatures0 hiddenFeatures1)
      <*> sample (LinearSpec hiddenFeatures1 outputFeatures)

mlp :: MLP -> Tensor -> Tensor
mlp MLP {..} =
  logSoftmax (Dim 1)
    . linear l2
    . relu
    . linear l1
    . relu
    . linear l0

trainLoop :: Optimizer o => Device -> DType -> MLP -> o -> ListT IO (Tensor, Tensor) -> IO MLP
trainLoop device' dtype' model optimizer = P.foldM step begin done . enumerateData
  where
    step :: MLP -> ((Tensor, Tensor), Int) -> IO MLP
    step model args = do
      let ((input, label), iter) = toLocalModel device' dtype' args
          loss = nllLoss' label $ mlp model input
      when (iter `mod` 50 == 0) $ do
        putStrLn $ "Iteration: " ++ show iter ++ " | Loss: " ++ show loss
      (newParam, _) <- runStep model optimizer loss 1e-3
      pure newParam
    done = pure
    begin = pure model

displayImages :: MLP -> (Tensor, Tensor) -> IO ()
displayImages model (testImg, testLabel) = do
  V.dispImage testImg
  putStrLn $ "Model        : " ++ (show . (argmax (Dim 1) RemoveDim . exp) $ mlp model testImg)
  putStrLn $ "Ground Truth : " ++ show testLabel

toLocalModel :: forall a. HasTypes a Tensor => Device -> DType -> a -> a
toLocalModel device' dtype' = over (types @Tensor @a) func
  where
    func tensor =
      let tensor' = toDevice device' tensor
          tensor'' =
            if isIntegral (dtype tensor')
              then tensor'
              else toType dtype' tensor'
       in tensor''

fromLocalModel :: forall a. HasTypes a Tensor => a -> a
fromLocalModel = over (types @Tensor @a) func
  where
    func tensor =
      let tensor' = toDevice (Device CPU 0) tensor
          tensor'' =
            if isIntegral (dtype tensor')
              then tensor'
              else toType Float tensor'
       in tensor''

main :: IO ()
main = do
  let device' = Device CUDA 0
      dtype' = Half
  (trainData, testData) <- initMnist "data"
  let trainMnist = V.MNIST {batchSize = 32, mnistData = trainData}
      testMnist = V.MNIST {batchSize = 1, mnistData = testData}
      spec = MLPSpec 784 64 32 10
      optimizer = GD
  init <- toLocalModel device' dtype' <$> sample spec
  model <- foldLoop init 5 $ \model _ ->
    runContT (streamFromMap (datasetOpts 2) trainMnist) $ trainLoop device' dtype' model optimizer . fst

  -- show test images + labels
  forM_ [0 .. 10] $ displayImages (fromLocalModel model) <=< getItem testMnist

  putStrLn "Done"
main

Iteration: 0 | Loss: Tensor Half []  6.4453   
Iteration: 50 | Loss: Tensor Half []  0.9331   
Iteration: 100 | Loss: Tensor Half []  1.3154   
Iteration: 150 | Loss: Tensor Half []  0.5312   
Iteration: 200 | Loss: Tensor Half []  0.5991   
Iteration: 250 | Loss: Tensor Half []  0.6724   
Iteration: 300 | Loss: Tensor Half []  0.3079   
Iteration: 350 | Loss: Tensor Half []  0.9829   
Iteration: 400 | Loss: Tensor Half []  0.3247   
Iteration: 450 | Loss: Tensor Half []  0.3933   
Iteration: 500 | Loss: Tensor Half []  0.6099   
Iteration: 550 | Loss: Tensor Half []  0.3398   
Iteration: 600 | Loss: Tensor Half []  0.4414   
Iteration: 650 | Loss: Tensor Half []  0.2700   
Iteration: 700 | Loss: Tensor Half []  0.2340   
Iteration: 750 | Loss: Tensor Half []  0.1393   
Iteration: 800 | Loss: Tensor Half []  0.2324   
Iteration: 850 | Loss: Tensor Half []  0.3386   
Iteration: 900 | Loss: Tensor Half []  0.1011   
Iteration: 950 | Loss: Tensor Half []  0.2455   
Iteration: 1000 | Loss: